In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from wordcloud import WordCloud, STOPWORDS
import dateparser
from collections import Counter
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Next we need to load our Chat.txt into Python and read it. We will do this using the function below:

In [2]:
with open('Chat.txt', "r", encoding='utf-8') as infile:
    output_Data = { 'DateTime': [], 'Name': [], 'Content': [] }
    for line in infile:
        matches = re.match(r'^(\d{1,2})\/(\d{1,2})\/(\d\d), (24:00|2[0-3]:[0-5][0-9]|[0-1][0-9]:[0-5][0-9]) - ((\S[^:]*?): )?(.*)$', line)
        if matches:
          output_Data['DateTime'].append(
            datetime(
              int(matches.group(3))+2000,
              int(matches.group(1)),
              int(matches.group(2)),
              hour=int(matches.group(4)[0:2]),
              minute=int(matches.group(4)[3:])
            ))
          output_Data['Name'].append(matches.group(6) or "{undefined}")
          output_Data['Content'].append(matches.group(7))

        elif len(output_Data['Content']) > 0:
          output_Data['Content'][-1] += "\n" + line[0:-1]


In [3]:
df = pd.DataFrame(output_Data)
df.head()

,DateTime,Name,Content
0,2019-11-15 06:31:00,{undefined},Messages to this group are now secured with en...
1,2019-11-11 09:41:00,{undefined},"Analoh RESAGRATIA created group ""RESA"""
2,2019-11-15 06:31:00,{undefined},You joined using this group's invite link
3,2019-11-15 07:06:00,AcidiQ,Good morning Fam.\nI just went through the Bud...
4,2019-11-15 07:29:00,+234 805 230 5080,Sounds great


Remove messages where Name is 'undefined' as these represent system messages.

In [ ]:
print("length of df before:{}".format(len(df)))
#Remove messages where Name is 'undefined' as these represent system messages.
df = df[~df["Name"].str.contains("undefined")]
print("length of df after:{}".format(len(df)))

In [ ]:
df.head(10)

In [ ]:
df[df["Content"].str.contains('\n')]

In [ ]:
df["Content"] = df["Content"].replace('\n', ' ', regex=True)

In [ ]:
df[df["Content"].str.contains('\n')]

 Create Columns for Date, Time, Word Count etc.

In [ ]:
df['Date'] = [datetime.date(d) for d in df['DateTime']] 
df["Date"]

In [ ]:
df['Time'] = [datetime.time(d) for d in df['DateTime']]
df["Time"].shape

In [ ]:
df['Hour'] = df.DateTime.dt.hour
df["Hour"]

In [ ]:
df['weekday'] = df['DateTime'].apply(lambda x: x.day_name())

In [ ]:
df['Word_Count'] = df['Content'].str.count(' ') + 1
df['Letter_Count'] = df['Content'].apply(lambda s : len(s))

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
#saving to csv format
df.to_csv("WhatsappChat1.csv")

We will clean our chat data by removing all messages in join and also removing all empty lines (lines that contain no message) by running the code below:



In [ ]:
#Remove new lines
chat = [line.strip() for line in chat]
chat[:10]
print("length of chat is:")
print(len(chat))

#Clean out the join notification lines
clean_chat = [line for line in chat if not "joined using this" in line]

#Further cleaning
#Remove empty lines
clean_chat = [line for line in clean_chat if len(line) > 1]
print("length of clean_chat is:")
print(len(clean_chat))


Next we do the same for messages that show members who left.


In [ ]:
#Drop 'left-ers'
left = [line for line in clean_chat if line.endswith("left")]
left

For the RESAGRATIA Whatsapp Chat, no members left in the period covered by this analysis as shown below:

In [ ]:
#Remove all that 'left'
#Clean out the left notification lines
clean_chat = [line for line in clean_chat if not line.endswith("left")]
print(len(clean_chat))

Next we will group all the lines in clean_chat into messages and store in a variable named msgs. Every message begins with a date e.g 12/12/19 and we will use this property in grouping. We will make use of the regex package in Python by running the code below:

In [ ]:
#Merge messages that belong together
msgs = [] #message container
pos = 0 #counter for position of msgs in the container
"""
Flow:
For every line, see if it matches the expression which is starting with the format "number(s)+slash" eg "12/"
If it does, it is a new line of conversion as they begin with dates, add it to msgs container
Else, it is a continuation of the previous line, add it to the previous line and append to msgs, then pop previous line.
"""
for line in clean_chat:
    if re.findall("\A\d+[/]", line):
        msgs.append(line)
        pos += 1
    else:
        take = msgs[pos-1] + ". " + line
        msgs.append(take)
        msgs.pop(pos-1)

len(msgs)

We have a total of 1472 unique messages (This should be different for your data).
Let’s look at the content of our msgs data:

In [ ]:
msgs[0:10]

Next we will need to extract Date, Time, Name and Message Content from our msgs data using the codes below:

In [ ]:
time = [msgs[i].split(',')[1].split('-')[0] for i in range(len(msgs))]
time = [s.strip(' ') for s in time] # Remove spacing
print("length of time is:")
print(len(time))
time[:10]

In [ ]:
date = [msgs[i].split(',')[0] for i in range(len(msgs))]
len(date)

In [ ]:
## Get name
name = [msgs[i].split('-')[1].split(':')[0] for i in range(len(msgs))]
len(name)

In [ ]:

content = []
for i in range(len(msgs)):
  try:
    content.append(msgs[i].split(':')[2])
  except IndexError:
    content.append('Missing Text')
len(content)

Now we can finally use the pandas library to merge our date, time, name and content data into a Dataframe named df using the code below:


In [ ]:
df = pd.DataFrame(list(zip(date, time, name, content)), columns = ['Date', 'Time', 'Name', 'Content'])
df

In [ ]:
df.shape

Look at the first two rows of data. The Content column shows “Missing Text.” Those are system messages and we’ll need to drop them. We can do this using:

In [ ]:
df = df[df["Content"]!='Missing Text']
df.reset_index(inplace=True, drop=True)
df

In [ ]:
df.shape

In [ ]:
df = df[df["Content"]!='Missing Text']
df.reset_index(inplace=True, drop=True)
df

We will create additional columns by taking advantage of built-in functions in pandas. First let us create a Datetime column by merging Date and Time columns and using the pd.to_datetime function:

In [ ]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['DateTime']

Using our newly created Datetime column we can create a weekday column showing days of the week

In [ ]:
df['weekday'] = df['DateTime'].apply(lambda x: x.day_name()) 

We will split our Content column to create new columns showing the number of letters and words contained in each message. We will call these columns Letter_count and Word_count respectively.

In [ ]:
df['Letter_Count'] = df['Content'].apply(lambda s : len(s))
df['Word_Count'] = df['Content'].apply(lambda s : len(s.split(' ')))




Next we will split our Time column to create a new column named Hour showing the hour of the day a message was sent. For example given 12:15, we will split the data before the colon (12) as this indicates hour.

In [ ]:
df['Hour'] = df['Time'].apply(lambda x : x.split(':')[0]) 
# The first token of a value in the Time Column contains the hour (Eg., "20" in "20:15")
df.head()

We can go on and create visuals using matplotlib and pandas but for this tutorial we will be creating only a WordCloud visual showing the most used words in the Group Chat. Word-clouds are used to perform high-level analysis and visualization of text data. The other visuals will be created using Power BI.
First we will need to further clean our data. Remember when we exported our data from Whatsapp, we selected the WITHOUT MEDIA option. Python indicates all those instances where the media was removed with a “<Media Omitted>” message. 
Also all messages that were deleted are indicated by the system message “This message was deleted.” 
NB: I did not remove deleted messages as I intend to use them in creating visuals.
Let’s go ahead and explore our data to confirm these presumptions.
We will use pandas built in function to list the count of unique messages in our Content column.

In [ ]:
df["Content"].value_counts().to_frame().head(5)

This would be slightly different for your data but you should see “<Media omitted>” and “This message was deleted.” “https” shows instances where a web url was shared in the group.
To create our WordCloud, we will make use of Vectorized String Operations in Python to remove instances where “<Media omitted>” and “This message was deleted” occur in our data. We will store this in a variable named “wordcloud_df.” We will still retain our original Dataframe df as we will use it to create visuals in Power BI.

In [ ]:
print("shape of df:")
print(df.shape)
#remove instances where "<Media omitted occur"
wordcloud_df = df[~df["Content"].str.contains("<Media omitted>")]
#remove instances where "This message was deleted occur"
wordcloud_df = wordcloud_df[~df["Content"].str.contains("This message was deleted")]
print("shape of wordcloud_df:")
print(wordcloud_df.shape)


We can now create our WordCloud visual using the wordcloud_df:
First we will create a function that generates the frequency of words in our data and returns the top 20 words.


In [ ]:

def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()

    #Print top 20 words
    word_freq[:20]
    
    return word_freq

gen_freq(wordcloud_df.Content.str)

Next we will create a function that removes punctuations and converts every word to lower-case to ensure uniformity.

In [ ]:
def clean_text(text):
    #Remove RT
    text = re.sub(r'RT', '', text)
    
    #Fix &
    text = re.sub(r'&amp;', '&', text)
    
    #Remove punctuations
    text = re.sub(r'[?!.;:,#@-]', '', text)
     
    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text

In [ ]:
#using stopwords we imported from wordcloud
stopwords = set(STOPWORDS)

Let’s go ahead and create our WordCloud visual using the functions we created above and the wordcloud package we imported earlier.

In [ ]:
%matplotlib inline
text = wordcloud_df.Content.apply(lambda x: clean_text(x))
word_freq = gen_freq(text.str)*100
word_freq = word_freq.drop(labels=STOPWORDS, errors='ignore')

#Generate word cloud
wc = WordCloud(width=450, height=400, max_words=300, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(14, 18))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.savefig('wordcloud.png', bbox_inches='tight')
plt.show()

We can superimpose the words onto a mask of any shape. For this tutorial, I will be using a mask of a chat icon. 
Using numpy we will download and save the mask to a variable named image_mask.

In [ ]:
from PIL import Image
image_mask = np.array(Image.open('Chaticon.jpeg'))

In [ ]:
#visualizing the mask
fig = plt.figure()
fig.set_figwidth(14) # set width
fig.set_figheight(18) # set height

plt.imshow(image_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis('off')
plt.show()

Now we can plot our WordCloud image superimposed on the image_mask.

In [ ]:
%matplotlib inline
text = wordcloud_df.Content.apply(lambda x: clean_text(x))
word_freq = gen_freq(text.str)*100
word_freq = word_freq.drop(labels=STOPWORDS, errors='ignore')

#Generate word cloud
wc = WordCloud(width=450, height=400, max_words=300, mask=image_mask, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(8, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.savefig('wordcloud1.png', bbox_inches='tight')
plt.show()

We can now save our Dataframe df in csv format by using pandas built in function. This csv will be used in creating our visuals in Power BI.

In [ ]:
#saving to csv format
df.to_csv("WhatsappChat.csv")